In [2]:
# Rumpfdatensatz'
import pandas as pd

In [7]:
df = pd.read_excel('test.xlsx', names=['signatur','jahr','datum','a2','hrsg','hrsgidn','titel','beschr','ort','ortidn','umfang','maße','quelle','bild'])
df.head()

,signatur,jahr,datum,a2,hrsg,hrsgidn,titel,beschr,ort,ortidn,umfang,maße,quelle,bild
0,2022/Arch/174,1880,17.02.1880,hand,Carl Ahlemann,NaN,"Brief an Jacob Isler & Co., Wohlen",Gedruckter Briefkopf mit Firmenansicht.,Leipzig,NaN,1 Blatt (1 Seite),XXX x XXX,Brief,x
1,2022/Arch/175,1921,1921-09-13 00:00:00,druck,Allgemeine Transportanlagen-Gesellschaft Masch...,NaN,"Brief an Herrn Rich. Rätzer, im Hause",Gedruckter Briefkopf mit Ansicht des Firmengel...,Leipzig,NaN,1 Blatt (2 Seiten),NaN,Brief,x
2,2022/Arch/176,1932,1932-04-27 00:00:00,masch,Allgemeine Transportanlagen-Gesellschaft Masch...,NaN,"Brief an Herrn Abt.Leiter Rätzer, im Hause",Gedruckter Briefkopf mit Ansicht des Firmengel...,Leipzig,NaN,1 Blatt (1 Seite),NaN,Brief,x
3,2022/Arch/177,1916,1916-04-20 00:00:00,druck,Berthold Bach,NaN,NaN,"Kohlen-Großhandlung, Versicherungen aller Art",Leipzig,NaN,NaN,NaN,Brief,NaN


In [15]:
def pica_schreibe(row):

    pica = f"""0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benuzen$bn
0503 Blatt$nb
0600 yy
1100 {row.jahr}
1110 $a{row.datum}$b{row.datum}$n{row.datum}$4ezth
1130 TB-papier
"""
    if pd.notna(row.bild):
        pica += f"1132 a1-aalog;a2-{row.a2};f1-text;f1-bild;f2-blatt;v-ganz\n"
    else:
        pica += f"1132 a1-aalog;a2-{row.a2};f1-text;f2-blatt;v-ganz\n"
    
    pica += f"""1500 /1ger
1700 /1XA-DXDE
"""
    if pd.notna(row.hrsgidn):
        pica +=f"3100 {row.hrsgidn}$BVerfasser$4aut\n"

    if pd.isna(row.hrsgidn) and pd.notna(row.hrsg):
        pica +=f"3100 {row.hrsg}$BVerfasser$4aut\n" 

    pica +=f"4000 {row.titel}"   

    if pd.notna(row.hrsg):
        pica +=f" / von {row.hrsg}"
    pica +=f"\n"
    pica +=f"""4046 {row.ort}
4060 {row.umfang}
"""
    
    if pd.notna(row.maße):
        werte = row.maße.split(' x ')
        pica += f"4062 {werte[0]} x {werte[1]} mm$b{werte[0]}$h{werte[1]}$4mesa\n"
    
    pica +=f"""4105 !1059146037!
4105 !1270063200!
4201 {row.beschr}
4700 |BSM|
"""

    if pd.notna(row.ortidn):
        pica +=f"5590 [Entstehungsort]\n5590 {row.ortidn}\n"
    else:
        pica +=f"5590 [Entstehungsort]\n5590 {row.ort}\n"
    
    pica +=f"""5591 [Objektgattung]
5591 {row.quelle}
5592 [Objektgattung]
5592 !042275423!
7001 x
4800 !009033645!
4821 $lLehmstedt, Mark, Leipzig$zErwerbung$D2022-09-29
"""
    signatur = row.signatur.split('/')
    pica +=f"6710 !1252833997!$l{signatur[2]}?n"
    pica +=f"6800 [Material]\n6800 !040445224!\n"
    pica +=f"7100 2022/Arch/{signatur[2]} @ m\n"
    pica +=f"7109 !!DBSM/S!! ; 2022/Arch/{signatur[2]}\n"
    pica +=f"8510 %ka\n"
    pica +=f"8598 [Z-Arch] 2022/00{signatur[2]}\n"



    return pica


In [16]:
for index, row in df.sample(3).iterrows():
    print(pica_schreibe(row))
    #test

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benuzen$bn
0503 Blatt$nb
0600 yy
1100 1880
1110 $a17.02.1880$b17.02.1880$n17.02.1880$4ezth
1130 TB-papier
1132 a1-aalog;a2-hand;f1-text;f1-bild;f2-blatt;v-ganz
1500 /1ger
1700 /1XA-DXDE
3100 Carl Ahlemann$BVerfasser$4aut
4000 Brief an Jacob Isler & Co., Wohlen / von Carl Ahlemann
4046 Leipzig
4060 1 Blatt (1 Seite)
4062 XXX x XXX mm$bXXX$hXXX$4mesa
4105 !1059146037!
4105 !1270063200!
4201 Gedruckter Briefkopf mit Firmenansicht.
4700 |BSM|
5590 [Entstehungsort]
5590 Leipzig
5591 [Objektgattung]
5591 Brief
5592 [Objektgattung]
5592 !042275423!
7001 x
4800 !009033645!
4821 $lLehmstedt, Mark, Leipzig$zErwerbung$D2022-09-29
6710 !1252833997!$l174?n6800 [Material]
6800 !040445224!
7100 2022/Arch/174 @ m
7109 !!DBSM/S!! ; 2022/Arch/174
8510 %ka
8598 [Z-Arch] 2022/00174

0500 Da
0501 Text$btxt
0502 ohne Hilfsmittel zu benuzen$bn
0503 Blatt$nb
0600 yy
1100 1916
1110 $a1916-04-20 00:00:00$b1916-04-20 00:00:00$n1916-04-20 00:00:00$4ezth
1130 TB-papi